In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rectal_Cancer"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Rectal_Cancer/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Rectal_Cancer/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Rectal_Cancer/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Rectal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# Step 1: Search for directories related to Rectal Cancer
import os

# List all directories in TCGA root directory
tcga_dirs = os.listdir(tcga_root_dir)

# Look for directories related to Rectal Cancer
matching_dirs = [dir_name for dir_name in tcga_dirs 
                if any(term in dir_name.lower() for term in 
                       ["rectal", "read", "coadread"])]

if not matching_dirs:
    print(f"No matching directory found for trait: {trait}")
    
    # Record that this trait is not available and exit
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
    print(f"Task marked as completed. {trait} is not directly represented in the TCGA dataset.")
else:
    # If we found matching directories
    print(f"Found matching directories: {matching_dirs}")
    
    # Select the most specific directory for rectal cancer
    if "TCGA_Rectal_Cancer_(READ)" in matching_dirs:
        selected_dir = "TCGA_Rectal_Cancer_(READ)"  # Choose the most specific match
    else:
        selected_dir = matching_dirs[0]  # Default to first match if specific one not found
    
    print(f"Selected directory: {selected_dir}")
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    
    # Step 2: Get file paths for clinical and genetic data
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    # Step 3: Load the files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # Step 4: Print column names of clinical data
    print("Clinical data columns:")
    print(clinical_df.columns.tolist())


Found matching directories: ['TCGA_Rectal_Cancer_(READ)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)']
Selected directory: TCGA_Rectal_Cancer_(READ)


Clinical data columns:
['AWG_MLH1_silencing', 'AWG_cancer_type_Oct62011', 'CDE_ID_3226963', 'CIMP', 'MSI_updated_Oct62011', '_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_READ', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'anatomic_neoplasm_subdivision', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'braf_gene_analysis_performed', 'braf_gene_analysis_result', 'circumferential_resection_margin', 'colon_polyps_present', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'followup_case_report_

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify candidate columns for age and gender
candidate_age_cols = ["age_at_initial_pathologic_diagnosis", "days_to_birth"]
candidate_gender_cols = ["gender"]

# Define cohort directory path
cohort_dir = os.path.join(tcga_root_dir, "TCGA_Rectal_Cancer_(READ)")

# Get clinical file path
clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)

# Read the clinical data
clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')

# Preview age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

# Preview gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

print("Age columns preview:")
print(age_preview)
print("\nGender columns preview:")
print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [57.0, 41.0, 41.0, 76.0, 48.0], 'days_to_birth': [-21098.0, -15082.0, -15082.0, -28119.0, -17759.0]}

Gender columns preview:
{'gender': ['MALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Evaluate age columns
age_col_candidates = {'age_at_initial_pathologic_diagnosis': [57.0, 41.0, 41.0, 76.0, 48.0], 
                      'days_to_birth': [-21098.0, -15082.0, -15082.0, -28119.0, -17759.0]}

# Evaluate gender columns
gender_col_candidates = {'gender': ['MALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}

# Select age column
# 'age_at_initial_pathologic_diagnosis' provides direct age values, while 'days_to_birth' requires conversion
# Both columns have no missing values in the preview, but 'age_at_initial_pathologic_diagnosis' is more straightforward
age_col = 'age_at_initial_pathologic_diagnosis'

# Select gender column
# Only one gender column available which contains valid values
gender_col = 'gender'

# Print the chosen columns
print(f"Chosen age column: {age_col}")
print(f"First 5 values of age column: {age_col_candidates[age_col]}")
print(f"Chosen gender column: {gender_col}")
print(f"First 5 values of gender column: {gender_col_candidates[gender_col]}")


Chosen age column: age_at_initial_pathologic_diagnosis
First 5 values of age column: [57.0, 41.0, 41.0, 76.0, 48.0]
Chosen gender column: gender
First 5 values of gender column: ['MALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Use the Rectal Cancer directory identified in Step 1
selected_dir = "TCGA_Rectal_Cancer_(READ)"
cohort_dir = os.path.join(tcga_root_dir, selected_dir)

# Get the file paths for clinical and genetic data
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Extract standardized clinical features using the provided trait variable
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait,  # Using the provided trait variable
    age_col=age_col, 
    gender_col=gender_col
)

# Save the clinical data to out_clinical_data_file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Saved clinical data with {len(clinical_features)} samples")

# Step 2: Normalize gene symbols in gene expression data
# Transpose to get genes as rows
gene_df = genetic_df

# Normalize gene symbols using NCBI Gene database synonyms
normalized_gene_df = normalize_gene_symbols_in_index(gene_df)
print(f"After normalization: {len(normalized_gene_df)} genes remaining")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Saved normalized gene expression data")

# Step 3: Link clinical and genetic data
# Merge clinical features with genetic expression data
linked_data = clinical_features.join(normalized_gene_df.T, how='inner')
print(f"Linked data shape: {linked_data.shape} (samples x features)")

# Step 4: Handle missing values
cleaned_data = handle_missing_values(linked_data, trait_col=trait)
print(f"After handling missing values, data shape: {cleaned_data.shape}")

# Step 5: Determine if trait or demographics are severely biased
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait=trait)

# Step 6: Validate data quality and save cohort information
note = "The dataset contains gene expression data along with clinical information for rectal cancer patients from TCGA."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=cleaned_data,
    note=note
)

# Step 7: Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Saved usable linked data to {out_data_file}")
else:
    print("Dataset was determined to be unusable and was not saved.")

Saved clinical data with 186 samples
After normalization: 19848 genes remaining


Saved normalized gene expression data
Linked data shape: (105, 19851) (samples x features)


After handling missing values, data shape: (105, 19851)
For the feature 'Rectal_Cancer', the least common label is '0' with 10 occurrences. This represents 9.52% of the dataset.
The distribution of the feature 'Rectal_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 54.0
  50% (Median): 63.0
  75%: 73.0
Min: 31.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 49 occurrences. This represents 46.67% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Saved usable linked data to ../../output/preprocess/Rectal_Cancer/TCGA.csv
